First, import the fish and modules.

In [1]:
import inspectable_fish
from mayavi import mlab
fish_labels = ['2015-07-11-1 Chena - Chinook Salmon (id #4)']
fishes = [inspectable_fish.InspectableFish(fish_label) for fish_label in fish_labels]
for fish in fishes:
    fish.load_state()

/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Then, assign movie_fish to the subject of the movie and plot its predicted detection field. Manually remove the toolbar at the top (right-clicking) and pan/zoom the scene into the best view for rotating around in the video. Then, hit "stop" on the popup window to stop interacting with it, which will stop this cell's execution and make executing the next one possible.

In [3]:
movie_fish = fishes[0]
fig3d = movie_fish.plot_predicted_detection_field(gridsize=70j, surfaces=False, velocities=False, stopButton=True)

/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/numpy/lib/function_base.py:2815: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/scipy/interpolate/interpolate.py:595: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/scipy/interpolate/interpolate.py:598: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


In [12]:
movie_fish = fishes[0]
movie_fish.cforager.print_parameters()

ff = movie_fish.cforager.get_parameter_named("flicker_frequency")
tau_0 = movie_fish.cforager.get_parameter_named("tau_0")
delta_0 = movie_fish.cforager.get_parameter_named("delta_0")

movie_fish.cforager.set_parameter(ff, 70)
movie_fish.cforager.set_parameter(tau_0, 1e-1)
movie_fish.cforager.set_parameter(delta_0, 1)
fig3d = movie_fish.plot_predicted_detection_field(gridsize=150j, show_fielddata=False, otf_exponent=0.3, surfaces=False, velocities=False, stopButton=True)

/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/numpy/lib/function_base.py:2815: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/scipy/interpolate/interpolate.py:595: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/scipy/interpolate/interpolate.py:598: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


In [19]:
import numpy as np
import matplotlib.pyplot as plt
from mayavi import mlab
from tvtk.util.ctf import PiecewiseFunction
import Fish3D
import scipy

movie_fish = fishes[0]


def plot_predicted_detection_field(self, gridsize=50j, stopButton=False, colorMax=None, **kwargs):
    def default_func(x, y, z):
        return self.cforager.relative_pursuits_by_position(0.01 * x, 0.01 * y, 0.01 * z)

    func = kwargs.get('func', default_func)
    vfunc = np.vectorize(func)
    r = 1.05 * 100 * self.cforager.get_max_radius()
    x, y, z = np.mgrid[-r:r:gridsize, -r:r:gridsize, -r:r:gridsize]
    s = vfunc(x, y, z)
    figname = "3D Fish Test Figure"
    myFig = mlab.figure(figure=figname, bgcolor=kwargs.get('bgcolor', (0, 0, 0)), size=(1024, 768))

    mlab.clf(myFig)
    head_position = np.array((0, 0, 0))
    tail_position = np.array((0, -self.fork_length_cm, 0))
    Fish3D.fish3D(head_position, tail_position, self.species, myFig, color=self.color,
                  world_vertical=np.array([0, 0, 1]))
    vmax = np.max(s) if colorMax is None else colorMax
    vol = mlab.pipeline.volume(mlab.pipeline.scalar_field(x, y, z, s), vmin=0.0, vmax=vmax)

    raw_predictions = s.flatten()
    predictions = sorted(raw_predictions[raw_predictions > 0])
    sums = [0]
    i = 0
    for i in range(len(predictions)):
        sums.append(sums[i] + predictions[i])
    sums = np.array(sums[1:])
    sums /= sums[-1]
    sums_interp = scipy.interpolate.interp1d(predictions, sums, fill_value="extrapolate", assume_sorted=True)

    def rel_pursuits_value_percentile(pct):
        def objfn(x):
            return abs(sums_interp(x) - pct)

        return scipy.optimize.minimize(objfn, [1], method='L-BFGS-B', bounds=[(min(predictions), max(predictions))]).x[
            0]

    def my_otf_value(v, exponent, divisor):
        return (v / divisor) ** exponent

    flat_s = s.flatten()
    nonzero_s = flat_s[flat_s > 0]
    otf = PiecewiseFunction()
    otf.add_point(0.0, 0.0)
    otf_exponent = kwargs.get("otf_exponent", 1.8)
    otf_divisor = kwargs.get("otf_divisor", 4)
    for v in np.linspace(0.05, 1.0, 40):
        otf_val = my_otf_value(v, otf_exponent, otf_divisor)
        otf.add_point(rel_pursuits_value_percentile(v), otf_val)
    # otf.add_point(0.0, 0.0)
    # otf.add_point(rel_pursuits_value_percentile(0.05), 0.006)
    # otf.add_point(rel_pursuits_value_percentile(0.15), 0.008)
    # otf.add_point(rel_pursuits_value_percentile(0.25), 0.013)
    # otf.add_point(rel_pursuits_value_percentile(0.35), 0.022)
    # otf.add_point(rel_pursuits_value_percentile(0.50), 0.03)
    # otf.add_point(rel_pursuits_value_percentile(0.65), 0.04)
    # otf.add_point(rel_pursuits_value_percentile(0.80), 0.05)
    # otf.add_point(rel_pursuits_value_percentile(0.90), 0.10)
    vol._otf = otf
    vol._volume_property.set_scalar_opacity(otf)

    if kwargs.get('colorbar', False):
        mlab.colorbar(vol)

    if kwargs.get('show_fielddata', True):
        (px, py, pz) = 100 * np.transpose(np.asarray(self.fielddata['detection_positions']))
        point_radius = 0.005 * self.fork_length_cm
        d = np.repeat(2 * point_radius, px.size)  # creates an array of point diameters
        mlab.points3d(py, -px, pz, d, color=kwargs.get('pointcolor', self.color), scale_factor=8, resolution=12,
                      opacity=1.0, figure=myFig)

    if kwargs.get('surfaces', True):
        sx = []
        sy = []
        sz = []
        bz = []
        rspace = np.linspace(-r, r, 100)
        for pbx in rspace:
            for pby in rspace:
                if np.sqrt(pbx * pbx + pby * pby) < 100 * self.cforager.get_max_radius():
                    sx.append(pbx)
                    sy.append(pby)
                    bz.append(100 * self.fielddata['bottom_z_m'])
                    sz.append(100 * self.fielddata['surface_z_m'])
        pts = mlab.points3d(sx, sy, bz)
        bottom_mesh = mlab.pipeline.delaunay2d(pts)
        mlab.pipeline.surface(bottom_mesh, opacity=0.4, color=(0.7, 0.4, 0.3))
        pts.remove()
        pts = mlab.points3d(sx, sy, sz)
        surface_mesh = mlab.pipeline.delaunay2d(pts)
        mlab.pipeline.surface(surface_mesh, opacity=0.1, color=(0.0, 1.0, 1.0))
        pts.remove()
    if kwargs.get('velocities', True):
        def velocity_function_unvectorized(vx, vy, vz, ):
            return (0, -100 * self.cforager.water_velocity(vz), 0)

        velocity_function = np.vectorize(velocity_function_unvectorized)
        velocities_z = 0.9 * np.linspace(100 * self.fielddata['bottom_z_m'], 100 * self.fielddata['surface_z_m'], 8)
        velocities_x = np.repeat(0, len(velocities_z))
        velocities_y = np.repeat(-self.fork_length_cm * 1.3, len(velocities_z))
        mlab.quiver3d(velocities_x, velocities_y, velocities_z, velocity_function, color=(1, 1, 0))

    mlab.show(stop=stopButton)
    return myFig

def depletion_func(x, y, z):
    return 17.75 - movie_fish.cforager.depleted_prey_concentration_total_energy(0.01 * x, 0.01 * y, 0.01 * z)

# 17.75 J/m3
# depleted by a max of 2.38 J/m3

fig3d = plot_predicted_detection_field(movie_fish,
                               gridsize=30j,
                               ofs_exponent=1.8,
                               otf_divisor=12,
                               colorbar=False,
                               stopButton=True,
                               surfaces=False,
                               bgcolor=(0,0,0),
                               colorMax=3,
                               show_fielddata=False,
                               func=depletion_func)

/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/numpy/lib/function_base.py:2815: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in greater
/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in greater
/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/scipy/interpolate/interpolate.py:595: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/scipy/interpolate/interpolate.py:598: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


Finally, execute this one -- after hitting 'stop' in the previous one -- to export the movie.

Run the result through Wondershare Pro (not Compressor) to convert it into a format that Quicktime and other players don't choke on. It can also play immediately in VLC.

In [20]:
import moviepy.video.VideoClip as vc
import os
VIDEO_EXPORT_PATH = "/Users/Jason/Desktop/"
fps = 60
revolution_time = 15
revolutions = 1
def make_frame(t):
    current_view = mlab.view(figure=fig3d)
    mlab.view(azimuth=90 + (t / revolution_time) * 360, elevation=current_view[1], distance=current_view[2], focalpoint=current_view[3], figure=fig3d)
    return mlab.screenshot(antialiased=True, figure=fig3d)
frame = make_frame(1)
rotating_movie = vc.VideoClip(make_frame, duration=revolution_time * revolutions - 1 / fps)
rotating_movie.write_videofile(VIDEO_EXPORT_PATH + movie_fish.label + ".mp4", fps=fps, codec='mpeg4', bitrate='8000k', audio=False)
mlab.close()
os.system('open "%s"' % VIDEO_EXPORT_PATH)

/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/mayavi-4.5.0-py3.5-macosx-10.6-x86_64.egg/mayavi/tools/camera.py:288: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if focalpoint is not None and not focalpoint == 'auto':


[MoviePy] >>>> Building video /Users/Jason/Desktop/2015-07-11-1 Chena - Chinook Salmon (id #4).mp4


[MoviePy] Writing video /Users/Jason/Desktop/2015-07-11-1 Chena - Chinook Salmon (id #4).mp4


  0%|          | 0/900 [00:00<?, ?it/s]

/Users/Jason/anaconda/envs/driftmodelenv/lib/python3.5/site-packages/mayavi-4.5.0-py3.5-macosx-10.6-x86_64.egg/mayavi/tools/camera.py:288: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if focalpoint is not None and not focalpoint == 'auto':


...

100%|█████████▉| 898/900 [35:19<00:04,  2.08s/it]

100%|█████████▉| 899/900 [35:21<00:02,  2.06s/it]

[MoviePy] Done.


[MoviePy] >>>> Video ready: /Users/Jason/Desktop/2015-07-11-1 Chena - Chinook Salmon (id #4).mp4 



0